## Introduction

Basic functionality of pymatgen is introduced.

Written using:
- pymatgen==2018.4.20
- python==3.6.3

Note that a API key of materials project may be required.

In [6]:
import pymatgen as mg

## 1. Basic Element, Specie and Composition objects

Pymatgen contains a set of core classes to represent an Element, Specie and Composition. These objects contains useful properties such as atomic mass, ionic radii, etc. These core classes are loaded by default with pymatgen. An Element can be created as follows:

### Element

In [19]:
c = mg.Element('C')
print('Atomic mass of Carbon is {}'.format(c.atomic_mass))  #by default the unit is atomic mass unit, amu
print('Its oxidation states are {}'.format(c.oxidation_states))
print('Its ionic radii is {}'.format(c.ionic_radii))
print('Its thermal conductivity is {}'.format(c.thermal_conductivity))

Atomic mass of Carbon is 12.0107 amu
Its oxidation states are (-4, -3, -2, -1, 1, 2, 3, 4)
Its ionic radii is {4: 0.3}
Its thermal conductivity is 140.0 W K^-1 m^-1


By examing the source code for pymatgen.core.periodic_table, we can find many other properties we can inquiry from the database.

Pymatgen comes with a complete system of managing units in pymatgen.core.unit. A Unit is a subclass of float that attaches units and handles conversions. For example,

In [18]:
print('Atomic mass of Carbon in kg is {}'.format(c.atomic_mass.to('kg')))

Atomic mass of Carbon in kg is 1.9944236247728e-26 kg


### Species

Species are like Elements, except they have an explicit oxidation state. They can be used wherever Element is used for the most part.

In [2]:
import pymatgen as mg
mn2 = mg.Specie("Mn", 2)
print(mn2.atomic_mass)
print(mn2.ionic_radius) # by default, the unit is angstrom
print(mn2.ionic_radius.to('bohr')) # convert ang to bohr

54.938045 amu
0.97 ang
1.8330343416940933 bohr


### Composition

A **Composition** is essentially an **immutable** mapping of Elements/Species with amounts, and useful properties like molecular weight, get_atomic_fraction, etc. Note that we can conveniently either use an Element/Specie object or a string as keys (this is a feature).

In [28]:
import pymatgen as mg
comp = mg.Composition("LaWN3")
# LaWN3 is identified as a good candidate of ferroelectric material, see Phys. Rev. B 95, 014111, 2017
print("Weight of LaWN3 is {}".format(comp.weight))
print("Amount of W in LaWN3 is {}".format(comp["W"]), "and N is {}".format(comp["N"]))
print("Weight fraction of La is {}".format(comp.get_wt_fraction('La')))

Weight of LaWN3 is 364.76557 amu
Amount of W in LaWN3 is 1.0 and N is 3.0
Weight fraction of La is 0.38080751426183124 


## 2. Lattice & Structure objects

### Lattice 

A Lattice represents a Bravais lattice. Convenience static functions are provided for the creation of common lattice types from a minimum number of arguments. 

In [16]:
import pymatgen as mg
# Creates cubic Lattice with lattice parameter 3.905
lattice = mg.Lattice.cubic(3.905)
print(type(lattice))
print(lattice.lengths_and_angles)
print(lattice.abc)

<class 'pymatgen.core.lattice.Lattice'>
((3.9049999999999998, 3.9049999999999998, 3.9049999999999998), (90.0, 90.0, 90.0))
(3.9049999999999998, 3.9049999999999998, 3.9049999999999998)


### Structure = (Lattice + Basis)

A Structure object represents a crystal structure (lattice + basis). A Structure is essentially a list of PeriodicSites with the same Lattice. Let us now create a CsCl structure.

#### A simple example of cubic CsCl structure

In [83]:
import pymatgen as mg
lattice = mg.Lattice.cubic(3.905)
structure = mg.Structure(lattice, ["Cs","Cl"], [[0,0,0],[0.5,0.5,0.5]])
print("Unit cell volume = {}".format(structure.volume)) # the default volume unit in pymatgen is ang^3

Unit cell volume = 59.54744262499999


In [85]:
# pymatgen provides us kinds of structure formats
# here, I export the file into POSCAR format that can be used by VASP.
structure.to(filename='IO-data/POSCAR-CsCl.vasp')
%cat IO-data/POSCAR-CsCl.vasp

Cs1 Cl1
1.0
3.905000 0.000000 0.000000
0.000000 3.905000 0.000000
0.000000 0.000000 3.905000
Cs Cl
1 1
direct
0.000000 0.000000 0.000000 Cs
0.500000 0.500000 0.500000 Cl


In [86]:
# CIF format and xyz format are also supported
# for example:
structure.to(filename='IO-data/CsCl.cif')
%cat IO-data/CsCl.cif

# generated using pymatgen
data_CsCl
_symmetry_space_group_name_H-M   'P 1'
_cell_length_a   3.90500000
_cell_length_b   3.90500000
_cell_length_c   3.90500000
_cell_angle_alpha   90.00000000
_cell_angle_beta   90.00000000
_cell_angle_gamma   90.00000000
_symmetry_Int_Tables_number   1
_chemical_formula_structural   CsCl
_chemical_formula_sum   'Cs1 Cl1'
_cell_volume   59.54744262
_cell_formula_units_Z   1
loop_
 _symmetry_equiv_pos_site_id
 _symmetry_equiv_pos_as_xyz
  1  'x, y, z'
loop_
 _atom_site_type_symbol
 _atom_site_label
 _atom_site_symmetry_multiplicity
 _atom_site_fract_x
 _atom_site_fract_y
 _atom_site_fract_z
 _atom_site_occupancy
  Cs  Cs1  1  0.000000  0.000000  0.000000  1
  Cl  Cl2  1  0.500000  0.500000  0.500000  1


#### A simple example of cubic BaTiO3 structure 

an ideal cubic BaTiO3 and its 2 * 2 * 2 superlattice

In [77]:
import pymatgen as mg
bto_lattice = mg.Lattice.cubic(3.905)
bto_structure = mg.Structure(bto_lattice,
                            ['Ba', 'Ti']+['O']*3,
                            [[0,0,0], [0.5,0.5,0.5], [0.5,0.5,0],[0.5,0,0.5],[0,0.5,0.5]])
# note that all modifications will be changed in palce.
bto_structure.to(filename='IO-data/POSCAR-cubic-BTO.vasp')
%cat IO-data/POSCAR-cubic-BTO.vasp
print('\n')
bto_structure.make_supercell([2, 2, 1])
bto_structure.to(filename='IO-data/POSCAR-cubic-BTO-40atom.vasp')
%cat IO-data/POSCAR-cubic-BTO-40atom.vasp

Ba1 Ti1 O3
1.0
3.905000 0.000000 0.000000
0.000000 3.905000 0.000000
0.000000 0.000000 3.905000
Ba Ti O
1 1 3
direct
0.000000 0.000000 0.000000 Ba
0.500000 0.500000 0.500000 Ti
0.500000 0.500000 0.000000 O
0.500000 0.000000 0.500000 O
0.000000 0.500000 0.500000 O


Ba4 Ti4 O12
1.0
7.810000 0.000000 0.000000
0.000000 7.810000 0.000000
0.000000 0.000000 3.905000
Ba Ti O
4 4 12
direct
0.000000 0.000000 0.000000 Ba
0.000000 0.500000 0.000000 Ba
0.500000 0.000000 0.000000 Ba
0.500000 0.500000 0.000000 Ba
0.250000 0.250000 0.500000 Ti
0.250000 0.750000 0.500000 Ti
0.750000 0.250000 0.500000 Ti
0.750000 0.750000 0.500000 Ti
0.250000 0.250000 0.000000 O
0.250000 0.750000 0.000000 O
0.750000 0.250000 0.000000 O
0.750000 0.750000 0.000000 O
0.250000 0.000000 0.500000 O
0.250000 0.500000 0.500000 O
0.750000 0.000000 0.500000 O
0.750000 0.500000 0.500000 O
0.000000 0.250000 0.500000 O
0.000000 0.750000 0.500000 O
0.500000 0.250000 0.500000 O
0.500000 0.750000 0.500000 O


dop a calcium ion into BaTiO3 superlattice and displace the titanum ions

**method-1**: delete the Ba ion first, and then add a new Ca ion to structure

In [80]:
copy_1 = bto_structure.copy()
del copy_1[0]
copy_1[-1]=("Ca",
       [0,0,0])
copy_1

Structure Summary
Lattice
    abc : 7.8099999999999996 7.8099999999999996 3.9049999999999998
 angles : 90.0 90.0 90.0
 volume : 238.18977049999995
      A : 7.8099999999999996 0.0 0.0
      B : 0.0 7.8099999999999996 0.0
      C : 0.0 0.0 3.9049999999999998
PeriodicSite: Ba (0.0000, 3.9050, 0.0000) [0.0000, 0.5000, 0.0000]
PeriodicSite: Ba (3.9050, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: Ba (3.9050, 3.9050, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Ti (1.9525, 1.9525, 1.9525) [0.2500, 0.2500, 0.5000]
PeriodicSite: Ti (1.9525, 5.8575, 1.9525) [0.2500, 0.7500, 0.5000]
PeriodicSite: Ti (5.8575, 1.9525, 1.9525) [0.7500, 0.2500, 0.5000]
PeriodicSite: Ti (5.8575, 5.8575, 1.9525) [0.7500, 0.7500, 0.5000]
PeriodicSite: O (1.9525, 1.9525, 0.0000) [0.2500, 0.2500, 0.0000]
PeriodicSite: O (1.9525, 5.8575, 0.0000) [0.2500, 0.7500, 0.0000]
PeriodicSite: O (5.8575, 1.9525, 0.0000) [0.7500, 0.2500, 0.0000]
PeriodicSite: O (5.8575, 5.8575, 0.0000) [0.7500, 0.7500, 0.0000]
PeriodicS

**method-2**

Replace the Ba ion with Ca ion directly at the specified site

In [82]:
copy_2 = bto_structure.copy()
copy_2[0] = 'Ca'
copy_2

Structure Summary
Lattice
    abc : 7.8099999999999996 7.8099999999999996 3.9049999999999998
 angles : 90.0 90.0 90.0
 volume : 238.18977049999995
      A : 7.8099999999999996 0.0 0.0
      B : 0.0 7.8099999999999996 0.0
      C : 0.0 0.0 3.9049999999999998
PeriodicSite: Ca (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Ba (0.0000, 3.9050, 0.0000) [0.0000, 0.5000, 0.0000]
PeriodicSite: Ba (3.9050, 0.0000, 0.0000) [0.5000, 0.0000, 0.0000]
PeriodicSite: Ba (3.9050, 3.9050, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: Ti (1.9525, 1.9525, 1.9525) [0.2500, 0.2500, 0.5000]
PeriodicSite: Ti (1.9525, 5.8575, 1.9525) [0.2500, 0.7500, 0.5000]
PeriodicSite: Ti (5.8575, 1.9525, 1.9525) [0.7500, 0.2500, 0.5000]
PeriodicSite: Ti (5.8575, 5.8575, 1.9525) [0.7500, 0.7500, 0.5000]
PeriodicSite: O (1.9525, 1.9525, 0.0000) [0.2500, 0.2500, 0.0000]
PeriodicSite: O (1.9525, 5.8575, 0.0000) [0.2500, 0.7500, 0.0000]
PeriodicSite: O (5.8575, 1.9525, 0.0000) [0.7500, 0.2500, 0.0000]
Periodic

### Other basic analyses for structure objects

Many propertie of structure object can be displayed, manipulated or revised by pymatgen.
For example, find the symmetry of structure, perform the fitting of structures.

#### symmetry of structure

Here, let's use cubic BaTiO3 as a simple example

In [105]:
import pymatgen as mg
bto_lattice = mg.Lattice.cubic(3.905)
bto_structure = mg.Structure(bto_lattice,
                            ['Ba', 'Ti']+['O']*3,
                            [[0,0,0], [0.5,0.5,0.5], [0.5,0.5005,0],[0.5,0,0.5],[0,0.5,0.5]])
#note that I shifted one of the O ion a little bit.

from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
# pymatgen.symmetry.analyzer acts as an interface of 'spglib' for pymatgen
bto_symmetry = SpacegroupAnalyzer(bto_structure,0.001) #if 0.1, symmetry will be Pm-3m
# Tolerance for symmetry finding. Defaults to 1e-3,
#             which is fairly strict and works well for properly refined
#             structures with atoms in the proper symmetry coordinates. For
#             structures with slight deviations from their proper atomic
#             positions (e.g., structures relaxed with electronic structure
#             codes), a looser tolerance of 0.1 (the value used in Materials
#             Project) is often needed.
print('The spacegroup of BaTiO3 is {}'.format(bto_symmetry.get_space_group_symbol()))

The spacegroup of BaTiO3 is Pmm2


#### fitting of structures

In [134]:
import pymatgen as mg
from pymatgen.analysis.structure_matcher import StructureMatcher
#Let's create two structures which are the same topologically, but with different elements, and one lattice is larger.
lattice = mg.Lattice.cubic(4.1)
s1 = mg.Structure(lattice, ["Cs", 'Cl'], [[0, 0, 0], [0.5, 0.5, 0.5]])
s2 = mg.Structure(mg.Lattice.cubic(4.02), ["Rb", "F"], [[0, 0, 0], [0.5, 0.5, 0.5]])

m = StructureMatcher()
print(m.fit_anonymous(s1, s2))

# def fit_anonymous(self, struct1, struct2, niggli=True):
#         """
#         Performs an anonymous fitting, which allows distinct species in one
#         structure to map to another. E.g., to compare if the Li2O and Na2O
#         structures are similar.

#         Args:
#             struct1 (Structure): 1st structure
#             struct2 (Structure): 2nd structure

#         Returns:
#             True/False: Whether a species mapping can map struct1 to stuct2
#         """


True
